# Load data and go to directory with python scripts

In [1]:
%cd 'scripts'
from dataLoader import *
from helper import *
import scipy.stats as st
from collections import Counter
import sklearn
from matplotlib import cm
import scipy
import  itertools
from datetime import datetime

from seaborn import clustermap
from scipy.cluster.hierarchy import linkage
from sklearn.linear_model import LogisticRegression
import os
import time
import pickle as pkl
import re
from matplotlib_venn import venn3, venn2
from statsmodels.stats.multitest import multipletests
dl = dataLoader(pt_perc = {'metabs': .25, '16s': .05, 'scfa': 0, 'toxin':0}, meas_thresh = 
                {'metabs': 0, '16s': 10, 'scfa': 0, 'toxin':0}, 
                var_perc = {'metabs': 15, '16s': 5, 'scfa': 0, 'toxin':0})


/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/Analyses/scripts


/Users/jendawk/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


# Define functions for doing over representation analysis

In [2]:
import math
def combination(n,k):
    return math.factorial(n)/(math.factorial(k)*math.factorial(n-k))

def ora(x, X):
    N = len(X)
    n = len(x)
    print('n=' + str(n))
    num_hits = {}
    num_hits_expected = {}
    prob = {}
    for group in np.unique(x):
        k = np.sum(np.array(x)==group)
        K = np.sum(np.array(X)==group)
        if K < 4 or k < 3 or group == 'None' or group == 'Partially Characterized Molecules':
            print(group + ': K='+str(K) + ', k=' + str(k))
            continue
        num_hits[group] = k
        num_hits_expected[group] = (K/N)*n
        prob[group] = (combination(K,k)*combination(N-K, n-k))/combination(N,n)
    if len(prob)==0:
        return None

    gps, pvals = zip(*prob.items())
    reject, corrected, a1, a2 = multipletests(list(pvals), alpha=.05, method='fdr_bh')
    fdr = {gp:corr for gp, corr in zip(gps, corrected)}
    out_df = pd.DataFrame({'# Hits':num_hits, '# Expected Hits': num_hits_expected, 'p-value': prob, 'FDR': fdr})
    return out_df.sort_values('p-value', ascending = True)

# Enrichment analysis for metabolites

In [4]:
# Set path to where univariate analysis results of metabolites is stored
path = '/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/Analyses/univariate_analysis/metabs/'
for file in os.listdir(path):
    if '.csv' not in file:
        continue
    if 'ranksum' in file:
        continue
        
    df = pd.read_csv(path + file, index_col = 0)
    mets = df.index.values[df['BH corrected']<= 0.05]
    
    for pathway in ['SUPER_PATHWAY','SUB_PATHWAY']:
        K = dl.col_mat_mets[pathway].loc[df.index.values]
        k = dl.col_mat_mets[pathway].loc[mets]
        print(file)
        print(pathway)
        print(ora(k,K))
        print('')

ttest_metabs0_1_Non-recurrer.csv
SUPER_PATHWAY
n=228
Energy: K=3, k=2
Partially Characterized Molecules: K=21, k=7
                        # Hits  # Expected Hits       p-value       FDR
Nucleotide                  28             12.9  8.033942e-07  0.000006
Amino Acid                  25             31.5  3.065450e-02  0.105318
Xenobiotics                 60             65.4  4.513633e-02  0.105318
Lipid                       89             88.8  6.462484e-02  0.113093
Cofactors and Vitamins       6              9.0  8.185609e-02  0.114599
Peptide                      6              8.1  1.206588e-01  0.140769
Carbohydrate                 5              5.1  2.104611e-01  0.210461

ttest_metabs0_1_Non-recurrer.csv
SUB_PATHWAY
n=228
Aminosugar Metabolism: K=3, k=1
Ceramides: K=2, k=2
Chemical: K=14, k=1
Dihydrosphingomyelins: K=1, k=1
Drug - Antibiotic: K=1, k=1
Drug - Other: K=1, k=1
Fatty Acid Metabolism (Acyl Carnitine, Long Chain Saturated): K=3, k=1
Fatty Acid Metabolism (Acyl Glu

# Enrichment analysis for ASVs

In [131]:
path = '/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/Analyses/univariate_analysis/16s/'
path_taxa = '/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/Analyses/scripts/inputs/'
taxa_rdp = pd.read_csv(path_taxa + 'dada2-taxonomy-rdp.csv', index_col = 0)
taxa_silva = pd.read_csv(path_taxa + 'dada2-taxonomy-silva.csv', index_col = 0)
for file in os.listdir(path):
    if '.csv' not in file:
        continue
        
    df = pd.read_csv(path + file, index_col = 0)
    for fname, taxa in zip(['rdp','silva'],[taxa_rdp, taxa_silva]):
        FAM = taxa[df.index.values].loc['Family'].dropna()
        GEN = taxa[df.index.values].loc['Genus'].dropna()

        mets = df.index.values[df['padj']<= 0.05]
        fam = taxa[mets].loc['Family'].dropna()
        gen = taxa[mets].loc['Genus'].dropna()
        print(file)
        print(fname)
        print('Family')
        print(ora(fam, FAM))
        print('Genus')
        print(ora(gen,GEN))
        print('')

deseq2_16sNon-recurrer1_2.csv
rdp
Family
n=12
Bacteroidaceae: K=14, k=1
Erysipelotrichaceae: K=12, k=1
Prevotellaceae: K=10, k=1
Ruminococcaceae: K=18, k=1
Veillonellaceae: K=21, k=1
Verrucomicrobiaceae: K=2, k=1
                 # Hits  # Expected Hits   p-value       FDR
Lachnospiraceae       6          3.06383  0.041147  0.041147
Genus
n=12
Akkermansia: K=2, k=1
Bacteroides: K=13, k=1
Clostridium_XVIII: K=3, k=1
Faecalibacterium: K=5, k=1
Fusicatenibacter: K=1, k=1
Prevotella: K=10, k=1
Roseburia: K=4, k=1
Ruminococcus2: K=3, k=1
Veillonella: K=9, k=1
                  # Hits  # Expected Hits   p-value       FDR
Clostridium_XlVa       3         1.023697  0.058486  0.058486

deseq2_16sNon-recurrer1_2.csv
silva
Family
n=12
Akkermansiaceae: K=2, k=1
Bacteroidaceae: K=13, k=1
Erysipelatoclostridiaceae: K=5, k=1
Prevotellaceae: K=10, k=1
Ruminococcaceae: K=13, k=1
Veillonellaceae: K=19, k=1
                 # Hits  # Expected Hits   p-value       FDR
Lachnospiraceae       6         3.128